In [1]:
from IPython.display import HTML, display
import csbgnpy.pd.io.sbgnml
import pypint
from tabulate import tabulate

This notebook has been executed using the docker image `pauleve/clockcycle:v1`

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

## Phases and markers

From [2 - Markers](2%20-%20Markers.ipynb)

In [3]:
%store -r clock_markers
%store -r clock_phases

To ids:

In [4]:
markers = {phase: [merge.get_entity(e, by_string=True).id for e in ms]
               for phase, ms in clock_markers.items()}

## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [5]:
%store -r inis

To ids:

In [6]:
inis = {code: [e.id for e in ini] for code, ini in inis.items()}

## Dynamical models

In [7]:
ans = [pypint.load(merge_file, initial_state=ini) for ini in inis.values()]

## A. Reach markers

In [8]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            res.append(an.reachability((marker, 1), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 3121
Computing for precursor state 3221
Computing for precursor state 1121
Computing for precursor state 1221
Computing for precursor state 3111
Computing for precursor state 3211
Computing for precursor state 1111
Computing for precursor state 1211
Computing for precursor state 3122
Computing for precursor state 1122
Computing for precursor state 3222
Computing for precursor state 1222
Computing for precursor state 3112
Computing for precursor state 1112
Computing for precursor state 3212
Computing for precursor state 1212
Computing for precursor state 2121
Computing for precursor state 2221
Computing for precursor state 2122
Computing for precursor state 2222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2211
Computing for precursor state 2212


In [9]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

Precursor state,RORG,SIRT1,ARNTL-CLOCK,PER-CRY
1111,1/1,1/1,1/1,1/1
1112,1/1,1/1,1/1,1/1
1121,1/1,1/1,1/1,1/1
1122,1/1,1/1,1/1,1/1
1211,1/1,1/1,1/1,1/1
1212,1/1,1/1,1/1,1/1
1221,1/1,1/1,1/1,1/1
1222,1/1,1/1,1/1,1/1
2111,0/1,0/1,0/1,0/1
2112,0/1,0/1,0/1,0/1


## B. Reach phases

In [10]:
results = []
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phase in markers:
        res = []
        for marker in markers[phase]:
            l = [(marker, 1)]
            for other_phase in markers:
                if other_phase != phase:
                    l += [(marker2, 0) for marker2 in markers[other_phase]]
            res.append(an.reachability(dict(l), reduce_for_goal=True, fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 3121
Computing for precursor state 3221
Computing for precursor state 1121
Computing for precursor state 1221
Computing for precursor state 3111
Computing for precursor state 3211
Computing for precursor state 1111
Computing for precursor state 1211
Computing for precursor state 3122
Computing for precursor state 1122
Computing for precursor state 3222
Computing for precursor state 1222
Computing for precursor state 3112
Computing for precursor state 1112
Computing for precursor state 3212
Computing for precursor state 1212
Computing for precursor state 2121
Computing for precursor state 2221
Computing for precursor state 2122
Computing for precursor state 2222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2211
Computing for precursor state 2212


In [11]:
display(HTML(tabulate(results, headers=["Precursor state"] + list(markers.keys()), tablefmt="html")))

Precursor state,RORG,SIRT1,ARNTL-CLOCK,PER-CRY
1111,1/1,1/1,1/1,1/1
1112,1/1,1/1,1/1,1/1
1121,1/1,1/1,1/1,1/1
1122,1/1,1/1,1/1,1/1
1211,1/1,1/1,1/1,1/1
1212,1/1,1/1,1/1,1/1
1221,1/1,1/1,1/1,1/1
1222,1/1,1/1,1/1,1/1
2111,0/1,0/1,0/1,0/1
2112,0/1,0/1,0/1,0/1


## C. Reach phases with previous phase disabled

In [12]:
results = []
couples = [(["RORG"], "SIRT1"), (["SIRT1"], "ARNTL-CLOCK"), (["RORG"], "ARNTL-CLOCK"), (["ARNTL-CLOCK"], "PER-CRY"), (["PER-CRY"], "RORG")]
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    for phases_disabled, phase in couples:
        res = []
        an_disabled = an.disable([(marker, 1) for phase_disabled in phases_disabled for marker in markers[phase_disabled]])
        for marker in markers[phase]:
            l = [(marker, 1)]
            for other_phase in markers:
                if other_phase != phase:
                    l += [(marker2, 0) for marker2 in markers[other_phase]]
            res.append(an_disabled.reachability(dict(l), fallback="mole"))
        res_ini.append("{}/{}".format(sum(res), len(res)))    
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 3121
Computing for precursor state 3221
Computing for precursor state 1121
Computing for precursor state 1221
Computing for precursor state 3111
Computing for precursor state 3211
Computing for precursor state 1111
Computing for precursor state 1211
Computing for precursor state 3122
Computing for precursor state 1122
Computing for precursor state 3222
Computing for precursor state 1222
Computing for precursor state 3112
Computing for precursor state 1112
Computing for precursor state 3212
Computing for precursor state 1212
Computing for precursor state 2121
Computing for precursor state 2221
Computing for precursor state 2122
Computing for precursor state 2222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2211
Computing for precursor state 2212


In [13]:
display(HTML(tabulate(results, headers=["Precursor state"] + ["{} -> {}".format(", ".join(phases_disabled), phase) for phases_disabled, phase in couples], tablefmt="html")))

Precursor state,RORG -> SIRT1,SIRT1 -> ARNTL-CLOCK,RORG -> ARNTL-CLOCK,ARNTL-CLOCK -> PER-CRY,PER-CRY -> RORG
1111,1/1,1/1,1/1,0/1,1/1
1112,1/1,1/1,1/1,0/1,1/1
1121,1/1,1/1,1/1,0/1,1/1
1122,1/1,1/1,1/1,0/1,1/1
1211,1/1,1/1,1/1,0/1,1/1
1212,1/1,1/1,1/1,0/1,1/1
1221,1/1,1/1,1/1,0/1,1/1
1222,1/1,1/1,1/1,0/1,1/1
2111,0/1,0/1,0/1,0/1,0/1
2112,0/1,0/1,0/1,0/1,0/1


## D. Reach phases sequentially n times (n > 1)

In [14]:
results = []
n = 4
for i, (code, ini) in enumerate(inis.items()):
    print("Computing for precursor state {}".format(code))
    an = ans[i]
    res_ini = [code]
    seq = []
    for phase in markers:
        l = [(marker, 1) for marker in markers[phase]]
        for other_phase in markers:
            if other_phase != phase:
                l += [(marker2, 0) for marker2 in markers[other_phase]]
        seq.append(dict(l))
    seq = seq * n
    res_ini.append(str(an.reachability(goal=pypint.Goal(*seq), reduce_for_goal=True, fallback="mole")))
    results.append(res_ini)
results = sorted(results, key=lambda x: x[0])

Computing for precursor state 3121
Computing for precursor state 3221
Computing for precursor state 1121
Computing for precursor state 1221
Computing for precursor state 3111
Computing for precursor state 3211
Computing for precursor state 1111
Computing for precursor state 1211
Computing for precursor state 3122
Computing for precursor state 1122
Computing for precursor state 3222
Computing for precursor state 1222
Computing for precursor state 3112
Computing for precursor state 1112
Computing for precursor state 3212
Computing for precursor state 1212
Computing for precursor state 2121
Computing for precursor state 2221
Computing for precursor state 2122
Computing for precursor state 2222
Computing for precursor state 2111
Computing for precursor state 2112
Computing for precursor state 2211
Computing for precursor state 2212


In [15]:
display(HTML(tabulate(results, headers=["Precursor state", "Reachable"], tablefmt="html")))

Precursor state,Reachable
1111,True
1112,True
1121,True
1122,True
1211,True
1212,True
1221,True
1222,True
2111,False
2112,False
